[「思わぬ大化け銘柄」を探すために使える指標とは？(ZUU online) - ニュース・コラム - Yahoo!ファイナンス](https://news.finance.yahoo.co.jp/detail/20171113-00000003-zuuonline-bus_all)
+ 株価がこれから上がる銘柄を探すには、いままでは出来高が少なかったものの直近で出来高が増えた銘柄、つまり「人気になったばかりの銘柄」を探す必要
+ 出来高の「良い増加」、「悪い増加」
    + 悪い増加:決算の悪化、粉飾決算や業務提携の解消など悪材料により売りたい人が多くなることによって起きる。株価がやすくても売りたいという人が増えるので、適正な価格で買いたい人が増えるまでは株価が下げていく可能性が高
    + 良い増加「業績上方修正」と「株主還元の向上」
+ 「普段から出来高が大きな銘柄」と「悪材料で出来高が増えた銘柄」を省く

+ 2018/10/25
    + 悪材料で出来高が増えた＝大きく下落して増えたのかも
    



In [ ]:
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.research import run_pipeline
from quantopian.pipeline import Pipeline,CustomFilter
from quantopian.pipeline.factors import CustomFactor
from quantopian.pipeline.filters import Q500US
from quantopian.pipeline.data import morningstar
import numpy as np
import pandas as pd 
import datetime 

from quantopian.pipeline.experimental import QTradableStocksUS

In [ ]:
class Liquidity(CustomFactor): 
    inputs = [USEquityPricing.volume, morningstar.valuation.shares_outstanding] 
    window_length = 1
    
    def compute(self, today, assets, out, volume, shares): 
        out[:] = volume[-1]/shares[-1]
        
class LiquidityMA30(CustomFactor): 
    inputs = [USEquityPricing.volume, morningstar.valuation.shares_outstanding] 
    window_length = 31
    
    def compute(self, today, assets, out, volume, shares): 
        out[:] = np.mean(volume[:-1], axis=0) / np.mean(shares[:-1], axis=0) 
        
class VolumeMA30(CustomFactor): 
    inputs = [USEquityPricing.volume,] 
    window_length = 31
    
    def compute(self, today, assets, out, volume): 
        out[:] = np.mean(volume[:-1], axis=0) 

class Diff5(CustomFactor): 
    inputs = [USEquityPricing.close,] 
    window_length = 5
    
    def compute(self, today, assets, out, price): 
        out[:] =price[-1]/price[-5]-1
        
        
        
def make_pipeline():
    base_universe = QTradableStocksUS()
    vol = USEquityPricing.volume
    price = USEquityPricing.close
    liquidity = Liquidity()
    liquidityMA30 =LiquidityMA30()
    volume30 = VolumeMA30()
    shares =  morningstar.valuation.shares_outstanding
    volume30_80p_90p = volume30.percentile_between(80, 90)
    diff_5 = Diff5()
    jump = diff_5 > 0.10
    
   # small_stocks = price <= 50
    
    pipe = Pipeline(
        columns = {
            'close': price.latest,
            'volume': vol.latest,
            'liquidity': liquidity,
            'liquidityMA30':liquidityMA30,
            'shares': shares.latest,
            'volume30':volume30,
            'diff_5':diff_5, 
        },
        screen = base_universe & volume30_80p_90p & jump)
    return pipe 

start_date = '2018-01-01'
spy = get_pricing("SPY", start_date=start_date,end_date= datetime.datetime.today())
end_date = spy.dropna().index[-1]

results = run_pipeline(make_pipeline(), 
                       start_date=start_date, 
                       end_date = end_date )


In [ ]:
results

In [ ]:

df = results.reset_index() 
df.plot(x="liquidity", y = "liquidityMA30", kind = 'scatter')



In [ ]:
df[(df.liquidity > 0.05) & (df.liquidity*2 > df.liquidityMA30)]




In [ ]:
targets = get_pricing(df[(df.liquidity > 0.05) & (df.liquidity*2 > df.liquidityMA30)].level_1.unique(),
           start_date = '2017-01-01',end_date = "2017-11-15", )

price_change = targets['price'].pct_change()
returns = targets['price'].pct_change().shift(-1)


In [ ]:
l = list()
for i,row in df[(df.liquidity > 0.05) & (df.liquidity*2 > df.liquidityMA30)].iterrows():
    date, symbol,liquidity, liquidityMA30 = row.level_0, row.level_1, row.liquidity, row.liquidityMA30
    l.append([liquidity, liquidityMA30, returns.loc[date, symbol], price_change.loc[date, symbol]])

In [ ]:
x = pd.DataFrame(l, columns = ["liquidity", "liquidityMA30","Returns","PriceChange"])
x[x.PriceChange < 0.1].plot(x="liquidity", y="Returns", kind="scatter", c ="PriceChange", s=10)#s=x["PriceChange"]*500


In [ ]:
pd.DataFrame(l, columns = ["liquidity", "liquidityMA30","Returns","PriceChange"]).sort_values(by="PriceChange",ascending=False)